# Bitmoji Parser

1. Read in exported coordinates from VIA
2. Go through and parse out rectangles (need to update to include other shapes)
3. Save them to `data/output`

In [1]:
from matplotlib import pyplot as plt
from glob import glob
import random as rng
import pandas as pd
import numpy as np
import json
import cv2

# Update these as necessary
VIA_PATH = '../data/via_coordinates.csv'
IMG_FOLDER = '../data/input/'
OUTPUT_FOLDER = '../data/output/'

via_output = pd.read_csv(VIA_PATH)
# Get the type out of the dictionary (should only be one?)
via_output['type'] = pd.json_normalize(via_output['region_attributes'].map(json.loads))

In [2]:
def parse_rectangle(img,coordinates):
    """
        Takes in a big image, returns that image cropped to the rectangle given by the 
        coordinates x,y,w,h
    """
    x,y,w,h = coordinates['x'],coordinates['y'],coordinates['width'],coordinates['height']
    section = img[y:y+h, x:x+w]
    return section
    
def parse_triangle(img,coordinates):
    """
        TODO
    """
    pass

def parse_oval(img,coordinates):
    """
        TODO
    """
    pass

def parse_polygon(img,coordinates):
    """
        TODO
    """
    pass
    
# Once the other functions are completed add them here
PARSE_MAP = { 
    'rect': parse_rectangle,
    'polygon': False,
    'triangle': False,
    'oval': False
}
    
def parse_image(row):
    # Parse region_shape_attributes column as a dictionary
    reg = json.loads(row['region_shape_attributes'])
    # Image meta data
    file_name,region_id,region_type = row['filename'],row['region_id'],row['type']
    # Right now the only shape I can parse are rectangles
    shape = reg.get('name')
    if not PARSE_MAP.get(shape):
        print(f"Parser can only handle rectangles. Got {reg.get('name')}.")
        return False
    parse_func = PARSE_MAP[shape]
    img = cv2.imread(f"{IMG_FOLDER}{file_name}", 1)
    section = parse_func(img,reg)
    # Get rid of the JPG at the end
    out_fname = file_name.split(".")[0]
    # Replace any whitespace or slashes with underscores
    region_type = region_type.replace(" ","_").replace("/","_")
    # Put the filename back together
    out_fname = f"{out_fname}_{region_type}-{region_id}.jpg"
    output_path = f"{OUTPUT_FOLDER}{out_fname}"
    # Write the image to the output folder
    cv2.imwrite(output_path, section)
    return True
    
via_output.apply(parse_image,axis=1)

Parser can only handle rectangles. Got polygon.


0      True
1      True
2      True
3      True
4      True
       ... 
105    True
106    True
107    True
108    True
109    True
Length: 110, dtype: bool